<a href="https://colab.research.google.com/github/prabal-1221/SimCLS-mini-project/blob/main/mini_project-2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#loading the datasets
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail','3.0.0')
print(dataset)

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
#training data
df_articles_train = dataset['train']['article']
df_highlights_train = dataset['train']['highlights']

#testing data
df_articles_test = dataset['test']['article']
df_highlights_test = dataset['test']['highlights']

In [ ]:
df_articles_train = df_articles_train[:100]
df_highlights_train = df_highlights_train[:100]

df_articles_test = df_articles_test[:100]
df_highlights_test = df_highlights_test[:100]

In [ ]:
#preprocessing the data
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def cleanup_sentences(text):
    stop_words = set(stopwords.words('english'))
    sentences = sent_tokenize(text)
    sentences_cleaned = []
    for sent in sentences:
        words = word_tokenize(sent)
        words = [w for w in words if w not in string.punctuation]
        words = [w for w in words if not w.lower() in stop_words]
        words = [w.lower() for w in words]
        sentences_cleaned.append(" ".join(words))
    return sentences_cleaned

#cleaning training data
for i in range(len(df_articles_train)):
  df_articles_train[i] = cleanup_sentences(df_articles_train[i])

#cleaning test data
for i in range(len(df_articles_test)):
  df_articles_test[i] = cleanup_sentences(df_articles_test[i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#importing t-5 model for beam search
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name_t5 = "t5-base"
tokenizer_t5 = AutoTokenizer.from_pretrained(model_name_t5)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(model_name_t5)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Set beam search hyperparameters
BEAMS = 5
MAX_LENGTH = 100
candidate_summary = [[] for i in range(len(df_articles_train))]

i=0
for article in df_articles_train:
  article = article[:500]
  input_ids = tokenizer_t5.encode(article, return_tensors="pt")
  output_ids = model_t5.generate(
    input_ids=input_ids,
    num_beams=BEAMS,
    max_length=MAX_LENGTH,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=BEAMS
    )
  for j, summary in enumerate(output_ids):
    summary = tokenizer_t5.decode(summary, skip_special_tokens=True)
    candidate_summary[i].append(summary)
  i += 1

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scores_train = [[] for i in range(len(df_articles_train))]

for i in range(len(df_highlights_train)):
  gold_summary = df_highlights_train[i][:500]
  for j in candidate_summary[i]:
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)
    scores_train[i].append(scorer.score(gold_summary, j))

# print(scores_train[0][0]['rouge1'].fmeasure)

for i in range(len(scores_train)):
  # print(scores_train[i])
  temp = []
  for j in scores_train[i]:
    temp.append(j['rouge1'].fmeasure)
  
  zipped_rouge1 = zip(temp, candidate_summary[i])
  candidate_summary[i] = [val for _, val in sorted(zipped_rouge1)]

  # zipped_rouge2 = zip(temp, candidate_summary[i])
  # candidate_summary[i] = [val for _, val in sorted(zipped_rouge2)]

In [ ]:
#importing sentence transformer for document embedding and summary embedding
from sentence_transformers import SentenceTransformer, util
model_sentence = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
import torch 
import torch.nn as nn

input_tensor = []
output_tensor = []

for i in range(len(df_articles_train)):
  doc_embed = model_sentence.encode(df_articles_train[i])
  doc_embed_tensor = torch.tensor(doc_embed, dtype = torch.float32)

  for summary in candidate_summary[i]:
    summary = model_sentence.encode(summary)

    summary_tensor = torch.tensor(summary, dtype = torch.float32)
    input_tensor.append(torch.cat((doc_embed_tensor, summary_tensor)))
    output_tensor.append(util.cos_sim(summary, doc_embed))

In [ ]:
#adding linear layer to learn the h function
n_in, n_out = len(input_tensor[0]), 1

model_seq = nn.Sequential(
   nn.Linear(n_in, n_out),
   nn.Sigmoid())

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model_seq.parameters(), lr = 0.01)
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   for i in range(len(input_tensor)):
    y_pred = model_seq(input_tensor[i])

    # Compute and print loss
    loss = criterion(y_pred, output_tensor[i][0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()

    # perform a backward pass (backpropagation)
    loss.backward()

    # Update the parameters
    optimizer.step()

In [ ]:
#testing data
# Set beam search hyperparameters
BEAMS = 5
MAX_LENGTH = 100
candidate_summary_test = [[] for i in range(len(df_articles_test))]

i=0
for article in df_articles_test:
  article = article[:500]
  input_ids = tokenizer_t5.encode(article, return_tensors="pt")
  output_ids = model_t5.generate(
    input_ids=input_ids,
    num_beams=BEAMS,
    max_length=MAX_LENGTH,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=BEAMS
    )
  for j, summary in enumerate(output_ids):
    summary = tokenizer_t5.decode(summary, skip_special_tokens=True)
    candidate_summary_test[i].append(summary)
  i += 1

input_tensor_test = []

for i in range(len(df_articles_test)):
  doc_embed = model_sentence.encode(df_articles_test[i])
  doc_embed_tensor = torch.tensor(doc_embed, dtype = torch.float32)

  for summary in candidate_summary[i]:
    summary = model_sentence.encode(summary)

    summary_tensor = torch.tensor(summary, dtype = torch.float32)
    input_tensor_test.append(torch.cat((doc_embed_tensor, summary_tensor)))

y_pred_test = []
count = 0
temp = []
summary_generated_test = []
max_index_test = []

for i in input_tensor_test:
  y_pred_test.append(model_seq(i))

for i in range(0,len(y_pred_test),5):
  temp = y_pred_test[i:i+5]
  temp = torch.tensor(temp, dtype=torch.float32)
  max_index = torch.argmax(temp, dim=0)
  max_index_test.append(max_index)

for i in range(len(candidate_summary_test)):
  summary_generated_test.append(candidate_summary_test[i][max_index_test[i]])

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scores = []

for i in range(len(summary_generated_test)):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)
    scores.append(scorer.score(df_highlights_test[i], summary_generated_test[i]))

avg_precision_rouge1 = 0
avg_recall_rouge1 = 0
avg_fmeasure_rouge1 = 0

avg_precision_rouge2 = 0
avg_recall_rouge2 = 0
avg_fmeasure_rouge2 = 0

avg_precision_rougeL = 0
avg_recall_rougeL = 0
avg_fmeasure_rougeL = 0
for i in range(len(scores)):
  avg_precision_rouge1 += scores[i]['rouge1'].precision
  avg_recall_rouge1 += scores[i]['rouge1'].recall
  avg_fmeasure_rouge1 += scores[i]['rouge1'].fmeasure

  avg_precision_rouge2 += scores[i]['rouge2'].precision
  avg_recall_rouge2 += scores[i]['rouge2'].recall
  avg_fmeasure_rouge2 += scores[i]['rouge2'].fmeasure

  avg_precision_rougeL += scores[i]['rougeL'].precision
  avg_recall_rougeL += scores[i]['rougeL'].recall
  avg_fmeasure_rougeL += scores[i]['rougeL'].fmeasure

avg_precision_rouge1 /= len(scores)
avg_recall_rouge1 /= len(scores)
avg_fmeasure_rouge1 /= len(scores)

avg_precision_rouge2 /= len(scores)
avg_recall_rouge2 /= len(scores)
avg_fmeasure_rouge2 /= len(scores)

avg_precision_rougeL /= len(scores)
avg_recall_rougeL /= len(scores)
avg_fmeasure_rougeL /= len(scores)

print(f"'rouge1': Score(precision={avg_precision_rouge1*100}, recall={avg_recall_rouge1*100}, fmeasure={avg_fmeasure_rouge1*100})")
print(f"'rouge2': Score(precision={avg_precision_rouge2*100}, recall={avg_recall_rouge2*100}, fmeasure={avg_fmeasure_rouge2*100})")
print(f"'rougeL': Score(precision={avg_precision_rougeL*100}, recall={avg_recall_rougeL*100}, fmeasure={avg_fmeasure_rougeL*100})")

'rouge1': Score(precision=28.819784615030986, recall=43.84107258732535, fmeasure=33.51148578319782)
'rouge2': Score(precision=11.640205647285525, recall=18.322103932880715, fmeasure=13.752838109459658)
'rougeL': Score(precision=20.239455662899555, recall=31.034725795719574, fmeasure=23.622621129932632)


In [ ]:
#prediction
sample_input = """LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed."""

BEAMS = 5
MAX_LENGTH = 100

sample_input = sample_input[:500]
# Generate candidate summaries using beam search
input_ids_pred = tokenizer_t5.encode(sample_input, return_tensors="pt")
output_ids_pred = model_t5.generate(
    input_ids=input_ids_pred,
    num_beams=BEAMS,
    max_length=MAX_LENGTH,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=BEAMS
)

input_tensor_pred = []

doc_embed_test = model_sentence.encode(sample_input)
doc_embed_tensor_test = torch.tensor(doc_embed_test, dtype = torch.float32)

for j, summary in enumerate(output_ids_pred):
  summary = tokenizer_t5.decode(summary, skip_special_tokens=True)
  summary = model_sentence.encode(summary)
  summary_tensor = torch.tensor(summary, dtype = torch.float32)
  input_tensor_pred.append(torch.cat((doc_embed_tensor, summary_tensor)))

y_pred_sample = []
for i in input_tensor_pred:
  y_pred_sample.append(model_seq(i))

y_pred_sample = torch.tensor(y_pred_sample, dtype=torch.float32)
max_index = torch.argmax(y_pred_sample, dim=0)
print(tokenizer_t5.decode(output_ids_pred[max_index], skip_special_tokens=True))

Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday. "I don't plan to be one of those people who, as gossip columnists around the world, will fritter my cash away on fast cars, drink and celebrity parties," the young actor says.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive

 cn-lab.ipynb		   'NN(using TenserFlow).ipynb'   text_summarizer_2.pt
'Colab Notebooks'	    Prabal-Maurya.gdoc		  text_summarizer.pt
 Documents		   'Resume (1).gdoc'		  Untitled0.ipynb
 Favourites		    Resume.gdoc			  Untitled1.ipynb
 mini-project-final.ipynb   RNN-Code_Example.ipynb	  Untitled2.ipynb
 mini-project.ipynb	    Sentiment_Analysis.ipynb


In [ ]:
model_save_name = 'text_summarizer_3.pt'
path = f"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model_seq.state_dict(), path)

In [ ]:
model_save_name = 'text_summarizer_3.pt'
path = f"/content/gdrive/My Drive/{model_save_name}"
model_seq.load_state_dict(torch.load(path))

<All keys matched successfully>